In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shapefile as shp
import seaborn as sns
import geopandas as gpd
import folium
import datetime
from shapely.geometry import Polygon

/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from plotly import __version__
import cufflinks as cf

In [3]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [4]:
%matplotlib inline

In [5]:
# Import dataset
# https://www.dataquest.io/blog/pandas-big-data/
df = pd.read_csv("Sampled_RH_dataset.csv")

# Filter dataset
list(df.columns)

['Unnamed: 0',
 'Trip ID',
 'Trip Start Timestamp',
 'Trip End Timestamp',
 'Trip Seconds',
 'Trip Miles',
 'Pickup Census Tract',
 'Dropoff Census Tract',
 'Pickup Community Area',
 'Dropoff Community Area',
 'Fare',
 'Tip',
 'Additional Charges',
 'Trip Total',
 'Shared Trip Authorized',
 'Trips Pooled',
 'Pickup Centroid Latitude',
 'Pickup Centroid Longitude',
 'Pickup Centroid Location',
 'Dropoff Centroid Latitude',
 'Dropoff Centroid Longitude',
 'Dropoff Centroid Location']

In [ ]:
df.info(memory_usage="deep")

# Data preprocessing (cleaning data)

In [6]:
df.drop(["Unnamed: 0", "Pickup Community Area", 
          "Dropoff Community Area", "Fare", "Tip",
          "Additional Charges"], axis=1)

,Trip ID,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Trip Total,Shared Trip Authorized,Trips Pooled,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location
0,c7661e85fe9b588a0dada78c493a52e3599e435f,02/04/2019 06:30:00 PM,02/04/2019 06:45:00 PM,1270.0,3.8,1.703184e+10,1.703124e+10,5.05,True,6,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.912364,-87.675063,POINT (-87.6750627566 41.9123643542)
1,c013b63cefbb16a8251586dd6ab04d31f8f42cb3,01/05/2019 02:45:00 AM,01/05/2019 03:00:00 AM,962.0,8.5,NaN,NaN,15.05,False,1,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.968069,-87.721559,POINT (-87.7215590627 41.968069)
2,c6036c4b6dd0cfa563823a82c4472201e8f4f044,01/19/2019 04:15:00 PM,01/19/2019 04:30:00 PM,353.0,0.8,1.703184e+10,1.703184e+10,7.55,False,1,41.853835,-87.693539,POINT (-87.6935387966 41.853834685),41.855501,-87.683227,POINT (-87.683227168 41.8555013528)
3,c30387fd0452828350411906001e3872954958e2,01/11/2019 03:00:00 PM,01/11/2019 03:30:00 PM,1221.0,6.2,1.703108e+10,1.703184e+10,15.05,False,1,41.899156,-87.626211,POINT (-87.6262105324 41.8991556134),41.841860,-87.639202,POINT (-87.6392016498 41.8418595653)
4,c63cbc0d9adf033435d4d712125057ae87b430dd,11/03/2018 12:30:00 PM,11/03/2018 12:30:00 PM,342.0,1.9,1.703103e+10,1.703106e+10,8.50,False,1,41.962179,-87.645379,POINT (-87.6453787624 41.962178629),41.949829,-87.643965,POINT (-87.6439653703 41.949829346)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149995,5311ff8f3a54a3137a83f88ccb449e4f824491de,03/22/2019 01:15:00 AM,03/22/2019 01:30:00 AM,667.0,7.5,NaN,NaN,12.55,False,1,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253)
1149996,4f98416947efec4e1e8ce1b74a2c5c11bc5c8674,02/15/2019 03:00:00 PM,02/15/2019 03:15:00 PM,495.0,1.2,1.703108e+10,1.703108e+10,7.55,False,1,41.905858,-87.630865,POINT (-87.6308650266 41.9058577688),41.899156,-87.626211,POINT (-87.6262105324 41.8991556134)
1149997,50b2d9374e6f27569744c2086c8a1fdb8cd48bc9,02/18/2019 08:30:00 AM,02/18/2019 09:00:00 AM,1620.0,6.5,1.703101e+10,1.703106e+10,7.50,True,3,42.009413,-87.663958,POINT (-87.6639582144 42.0094125471),41.938391,-87.638575,POINT (-87.6385749205 41.9383912577)
1149998,504ef8e388b974ad38441554d47e1b2e9eb41842,11/10/2018 06:15:00 AM,11/10/2018 06:30:00 AM,727.0,6.8,1.703138e+10,1.703128e+10,10.00,True,2,41.808512,-87.613999,POINT (-87.6139994504 41.808511513),41.879255,-87.642649,POINT (-87.642648998 41.8792550844)


In [ ]:
# Count missing values
# df.isnull().sum()

# Summary statistics
df.describe().astype(int)

In [7]:
# Approach 1 to month, day, hour variables out of timestamp.
# Step1: convert pickup and dropoff timestamp from objects into 
# datetime format, making analysis of date and time easier.
df["Trip Start Timestamp"] = pd.to_datetime(df["Trip Start Timestamp"])
df["Trip End Timestamp"] = pd.to_datetime(df["Trip End Timestamp"])

In [8]:
# Extract month and day from the pickup and dropoff variables.
df["Pickup_month"] = pd.DatetimeIndex(df["Trip Start Timestamp"]).month
df["Dropoff_month"] = pd.DatetimeIndex(df["Trip End Timestamp"]).month
df["Pickup_day"] = pd.DatetimeIndex(df["Trip Start Timestamp"]).day
df["Dropoff_day"] = pd.DatetimeIndex(df["Trip End Timestamp"]).day
df["Pickup_hour"] = pd.DatetimeIndex(df["Trip Start Timestamp"]).hour
df["Dropoff_hour"] = pd.DatetimeIndex(df["Trip End Timestamp"]).hour

In [9]:
# Distribution of time-series in week of the year, and weekdays.
df["Pickup_day_of_week"] = pd.DatetimeIndex(df["Trip Start Timestamp"]).dayofweek
df["Dropoff_day_of_week"] = pd.DatetimeIndex(df["Trip End Timestamp"]).dayofweek

df["Pickup_week_of_year"] = pd.DatetimeIndex(df["Trip Start Timestamp"]).week
df["Dropoff_week_of_year"] = pd.DatetimeIndex(df["Trip End Timestamp"]).week

In [ ]:
df["Trip Seconds"].describe().astype(int)

In [ ]:
# Calculate 1-100th percentile of data.
for i in range(1,100,10):
    var = df["Trip Seconds"].values
    var = np.sort(var,axis=None)
    print("{} percentile value is {}".format(i, var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
# Produce subplots related to trip features to show postive skew of data.
# Creates histograms to see distribution but also outliers.
df.hist(["Trip Seconds", "Trip Miles", "Trip Total", "Trips Pooled"], bins=40)
plt.tight_layout()
#plt.savefig("Distribution time, distance, fare total", dpi=200)

In [ ]:
# Create a correlation matrix of relevant variables
smalldf = df[["Trip Seconds", "Trip Miles", "Trip Total", "Trips Pooled"]]
smalldf.corr(method="pearson")

In [ ]:
# Line of code showing Trips longer than X seconds.
df.loc[smalldf['Trip Seconds'] > 4000]

In [ ]:
df["Trip Seconds"].plot.hist(bins=50) # Create a histogram to see distribution but also outliers.
plt.title("Trip duration in seconds")
# plt.savefig("Trip duration in seconds", dpi=100)

In [ ]:
#Find anomalies according to 3*STD and put them in a list.
data_std = np.std(df["Trips Pooled"])
data_mean = np.mean(df["Trips Pooled"])
anomaly_cutoff = data_std * 3

upper = data_mean + anomaly_cutoff
lower = data_mean - anomaly_cutoff

outliers = [x for x in data["Trips Pooled"] if x > upper or x < lower]
len(outliers)

In [ ]:
df.plot(x ='Trip Miles', y='Trip Seconds', kind = 'scatter')
plt.title("Time x Distance")
# plt.savefig("Time x Distance", dpi=100)

In [ ]:
# Check whether Trip Total are correlated to Trips Pooled.
df.plot(x ='Trips Pooled', y='Trip Total', kind = 'scatter')
#plt.savefig("Trip Total x Trips Pooled", dpi=100)
#data["Trip Total"].corr(data["Trips Pooled"])

In [ ]:
len([x for x in df["Trip Seconds"] if x > 5000])

In [10]:
# Create an average speed variable to identify rides where time and
# distance don't match up.
df["Average Speed"] = (df["Trip Miles"]/(df["Trip Seconds"])*3600)


# Optimize dtypes from float64 and int64 to float32 and int32

In [ ]:
# First , change columns with dtype float64 to float32
df[["Trip Seconds", "Trip Miles", "Pickup Census Tract", 
     "Dropoff Census Tract", "Trip Total", "Pickup Centroid Latitude", 
     "Pickup Centroid Longitude", "Dropoff Centroid Latitude", 
    "Dropoff Centroid Longitude", "Average Speed"]] = df[["Trip Seconds", "Trip Miles", "Pickup Census Tract", 
                                                          "Dropoff Census Tract", "Trip Total", "Pickup Centroid Latitude", "Pickup Centroid Longitude", 
                                                          "Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Average Speed"]].astype("float32")

# Second, change variables with dtype int64 to int32.
df[["Trips Pooled", "Pickup_month", "Dropoff_month",
      "Pickup_day", "Dropoff_day", "Pickup_hour", 
      "Dropoff_hour"]] = df[["Trips Pooled", "Pickup_month", "Dropoff_month",
                                        "Pickup_day", "Dropoff_day", "Pickup_hour", 
                                        "Dropoff_hour",]].astype("int32")


# Change dtype of "Pickup_day_of_week" and "Dropoff_day_of_week"
# and "Pickup_week_of_year" and "Dropoff_week_of_year"
# from int64 to int32.
df[["Pickup_day_of_week", "Dropoff_day_of_week",
    "Pickup_week_of_year", "Dropoff_week_of_year"]] = df[["Pickup_day_of_week", "Dropoff_day_of_week",
                                                            "Pickup_week_of_year", "Dropoff_week_of_year"]].astype("int32")


In [ ]:
df.info(memory_usage="deep")

In [ ]:
df["Average Speed"].plot.box()

In [ ]:
for i in range(90,100,1):
    var = df["Average Speed"].values
    var = np.sort(var,axis=None)
    print("{} percentile value is {}".format(i, var[int(len(var)*(float(i)/100))]))
print ("100 percentile value is ",var[-1])

In [ ]:
len([x for x in df["Average Speed"] if x < 3])

In [11]:
# Slice out rows with certain values.
large_time = df["Trip Seconds"] > 10800 # Create Boolean variable
short_time = df["Trip Seconds"] < 60
large_miles = df["Trip Miles"] > 200
short_miles = df["Trip Miles"] < 0.3 # This is 0.3 miles = 0.5 km.
large_fare = df["Trip Total"] > 200
low_fare = df["Trip Total"] < 2
low_speed = df["Average Speed"] < 3 # This is 3 miles per hour.
high_speed = df["Average Speed"] > 60
outlier_tripsecmilesfare = df[large_time | short_time | 
                                large_miles | short_miles| 
                                large_fare | low_fare |
                                high_speed | low_speed].index # Use Boolean variable to filter dataframe.

df.drop(outlier_tripsecmilesfare, inplace=True, axis=0)

In [ ]:
df.describe().astype(int)

In [ ]:
df["Average Speed"].describe().astype(int)

# Visualizing pickup- and dropoff locations

In [14]:
# Create an interactive base map.
# Default_location: start location based on coordinates.
# Zoom_start: defines default magnifing level.
def generateBaseMap(default_location=[41.881832, -87.623177], default_zoom_start=10):
    bmap = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return bmap

In [15]:
bmap = generateBaseMap()
bmap

In [13]:
# Plot heatmap.
from folium.plugins import HeatMap

df['Count'] = 1
bmap = generateBaseMap()
#Pickup coordinates
#HeatMap(data=data[["Pickup Centroid Latitude", "Pickup Centroid Longitude", "count"]].groupby(["Pickup Centroid Latitude", "Pickup Centroid Longitude"]).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(bmap)

#Dropoff coordinates
HeatMap(data=df[["Dropoff Centroid Latitude", "Dropoff Centroid Longitude", "Count"]].groupby(["Dropoff Centroid Latitude", "Dropoff Centroid Longitude"]).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(bmap)



NameError: name 'generateBaseMap' is not defined

In [ ]:
bmap

In [ ]:
# Heatmaps with a time dynamic component part 1.
pickup_hourly = []

for hour in df["Pickup_hour"].sort_values().unique():
    pickup_hourly.append(df.loc[df["Pickup_hour"] == hour, ["Pickup Centroid Latitude", "Pickup Centroid Longitude", "Count"]].groupby(["Pickup Centroid Latitude", "Pickup Centroid Longitude"]).sum().reset_index().values.tolist())


In [ ]:
# Heatmaps with a time dynamic component part 2.
from folium.plugins import HeatMapWithTime

bmap = generateBaseMap(default_zoom_start=11)
HeatMapWithTime(pickup_hourly, radius=5, gradient={0.2: 'blue', 0.3: 'lime', 0.5: 'orange', 0.6: 'red'}, min_opacity=0.5, max_opacity=0.8, use_local_extrema=True).add_to(bmap)
bmap

# Need to make this HOURLY --> first make an hour and 15th minute feature.

# Mapping Chicago with Geopandas from scratch with Cencus tracts.

# Time-series analysis on a monthly, weekly, and hourly basis (also spatial analysis)

In [ ]:
df.info()

In [ ]:
df["Pickup_day_of_week"].value_counts()
# Most pickups are on Friday and Saturday. Monday has the least.

In [ ]:
df["Dropoff_day_of_week"].value_counts()
# Most Dropoffs are on Friday and Saturday. Monday has the least.

In [ ]:
# Visual time-series of trips per day of week.
df["Pickup_day_of_week"].plot.bar()

plt.title("Picks distribution by day")
# plt.savefig("Trip duration in seconds", dpi=100)

In [ ]:
# Create function that makes a distribution of pickups and dropoffs
# during the day.
# Here, time is represented into 4 time parts: early morning (4AM - 10 AM),
# afternoon (10AM - 4PM), late afternoon/evening (4PM - 10PM)
# and late night (10PM - 4AM)

def timezone(x):
    if x>= datetime.time(6, 0, 1) and x <= datetime.time(12, 0, 0):
        return "Early Morning"
    if x>= datetime.time(12, 0, 1) and x <= datetime.time(18, 0, 0):
        return "Midday"
    if x>= datetime.time(18, 0, 1) and x <= datetime.time(24, 0, 0):
        return "Evening"
    if x>= datetime.time(24, 0, 1) and x <= datetime.time(6, 0, 0):
        return "Night"
    
df["Pickup_timezone"] = df["Start Time"].apply(lambda x: timezone())

In [14]:
# Write smaller csv
df['Count'] = 1

df[["Trip ID", "Trip Start Timestamp", "Trip End Timestamp", 
    "Trip Seconds", "Trip Miles", "Pickup Census Tract", 
    "Dropoff Census Tract", "Trip Total", "Shared Trip Authorized",
    "Trips Pooled", "Pickup Centroid Latitude", "Pickup Centroid Longitude",
    "Pickup Centroid Location", "Dropoff Centroid Latitude", 
    "Dropoff Centroid Longitude", "Dropoff Centroid Location",
    "Pickup_month", "Dropoff_month", "Pickup_day", "Dropoff_day",
    "Pickup_hour", "Dropoff_hour", "Pickup_day_of_week",
    "Dropoff_day_of_week", "Pickup_week_of_year", "Dropoff_week_of_year",
    "Average Speed", "Count"]].to_csv('RH_dataset_clean_w_censustracts.csv', index="False")

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1142290 entries, 0 to 1149999
Data columns (total 34 columns):
 #   Column                      Non-Null Count    Dtype         
---  ------                      --------------    -----         
 0   Unnamed: 0                  1142290 non-null  int64         
 1   Trip ID                     1142290 non-null  object        
 2   Trip Start Timestamp        1142290 non-null  datetime64[ns]
 3   Trip End Timestamp          1142290 non-null  datetime64[ns]
 4   Trip Seconds                1140087 non-null  float64       
 5   Trip Miles                  1142244 non-null  float64       
 6   Pickup Census Tract         819473 non-null   float64       
 7   Dropoff Census Tract        814967 non-null   float64       
 8   Pickup Community Area       1069403 non-null  float64       
 9   Dropoff Community Area      1061340 non-null  float64       
 10  Fare                        1142288 non-null  float64       
 11  Tip                     